In [6]:
%run my_import

#uri = socket.gethostname()+".local:5555"
#uri = "localhost:5555"
#uri = "nuc-i7.local:5555"
uri = "com-exp.local:5555"
#uri = "advantech.local:5555"
#uri = "192.168.1.100:5555"

io = set_uri(uri)
io.debug = False
scan_ids = list(reply_cmd(master_cmd_get_slave_descr).keys())
print('Found ESC ids {}'.format(scan_ids))


Python path : /home/amargan/work/anaconda3/envs/repl/bin/python
Python version : 3.9.13 | packaged by conda-forge | (main, May 27 2022, 16:56:21) 
[GCC 10.3.0]
new uri com-exp.local:5555
Found ESC ids [696]


In [51]:
from simple_pid import PID

ch_map = {
    0: 'force_x',
    1: 'force_y',
    2: 'force_z',
    3: 'torque_x',
    4: 'torque_y',
    5: 'torque_z',
}

def read_channel(bid:int, ch:int)->dict:
    write_sdo({'adc_idx':ch},[bid])
    reply_cmd(ctrl_cmd_get_adc.set_bid(bid))
    return read_sdo(['adc_conv','adc_raw'],[bid])[bid]

def read_channel_dac_offset(bid:int, ch:int, offset:int) :
    write_sdo({'adc_idx': ch,'dac_offset':offset},[bid])
    reply_cmd(ctrl_cmd_set_dac.set_bid(bid))
    reply_cmd(ctrl_cmd_get_adc.set_bid(bid))
    return read_sdo(['adc_conv','adc_raw'],[bid])[bid]

def read_channel_adc_offset(bid:int, ch:int, offset:float) :
    write_sdo({'adc_idx': ch,'adc_conv_offset':offset},[bid])
    # call Set_adc_offset_flash :
    #  - sdo.flash.adc_conv_offs[sdo.ram.adc_idx] = sdo.ram.adc_conv_offset;
    ctrl_status_cmd(0x00AF, bid)
    reply_cmd(ctrl_cmd_get_adc.set_bid(bid))
    return read_sdo([ch_map[ch],'adc_conv','adc_raw'],[bid])[bid]

def reset_DAC() :
    for ch in range(6) :
        ret = read_channel_offset(iD,ch,0x7FFF)
        print(ret)

def tune_adc_chan(bid:int, adc_idx:int) :
    dac_offset = 0x7FFF
    hister = 5
    tuned = False
    while ( not tuned ):
        yaml_msg = read_channel_dac_offset(bid,adc_idx,dac_offset)
        adc_raw = yaml_msg['adc_raw']
        adc_conv = yaml_msg['adc_conv']
        if ( adc_raw > 0x1FFF+hister ) :
            if ( dac_offset < 0xFFFF ) :
                # old pcb see github ticket #56
                #dac_offset += 2
                dac_offset -= 2
        elif ( adc_raw < 0x1FFF-hister ) :
            if ( dac_offset > 0 ) :
                # old pcb see github ticket #56
                #dac_offset -= 2
                dac_offset += 2
        else : 
            tuned = True
        time.sleep(0.001)
        yield(tuned, adc_raw, adc_conv, dac_offset)
        
def tune_adc_offset(bid:int, adc_idx:int, set_point:float) :
    adc_offset = 0.0
    hister = 0.5
    tuned = False
    while ( not tuned ):
        yaml_msg = read_channel_adc_offset(bid,adc_idx,adc_offset)
        adc_raw = yaml_msg['adc_raw']
        adc_conv = yaml_msg['adc_conv']
        adc = yaml_msg[ch_map[adc_idx]]
        adc_offset = adc_conv
        if abs(adc) < hister :
            tuned = True
        time.sleep(0.001)
        yield(tuned, adc_raw, adc_conv, adc_offset)

In [8]:
iD = scan_ids[0]
sdo_objd = reply_cmd(SdoInfo(u'SDO_OBJD').set_bid(iD))
sdo_names = reply_cmd(SdoInfo(u'SDO_NAME').set_bid(iD))
sdo_dict = reply_cmd(SdoCmd(rd_sdo=sdo_names,wr_sdo={}).set_bid(iD))
sdo_dict

{'adc_conv': 0.038125,
 'adc_conv_coeff': 1.0,
 'adc_conv_offset': 0.0,
 'adc_idx': 0,
 'adc_raw': 8441,
 'analog_sample_freq_HZ': 20000,
 'aux_get': 4568.0,
 'aux_set': 4568.0,
 'board_id': 696,
 'control': 0.0,
 'ctrl_status_cmd': 173,
 'ctrl_status_cmd_ack': 30893,
 'dac_offset': 33223,
 'dac_tune_bitflag': 0,
 'dac_tune_hysteresis': 5,
 'fault_ack': 0,
 'flash_params_cmd': 86,
 'flash_params_cmd_ack': 30806,
 'force_x': 0.038125,
 'force_y': 0.015708,
 'force_z': -0.001677,
 'fw_ver': 23041718,
 'module_params': 0,
 'offset': 0.0,
 'op_idx_ack': 257,
 'op_idx_aux': 257,
 'rtt': 4568.0,
 'smpl_num_pow2': 5,
 'status': 0.0,
 'temperature': 0.0,
 'torque_x': 0.047885,
 'torque_y': -0.015097,
 'torque_z': 0.046665,
 'ts': 4568.0}

In [9]:
flash_param = ['fw_ver','analog_sample_freq_HZ','dac_tune_bitflag',
               'smpl_num_pow2','dac_tune_hysteresis','adc_conv_coeff']
sdo_filter(flash_param, sdo_dict)

{'fw_ver': 23041718,
 'analog_sample_freq_HZ': 20000,
 'dac_tune_bitflag': 0,
 'smpl_num_pow2': 5,
 'dac_tune_hysteresis': 5,
 'adc_conv_coeff': 1.0}

In [10]:
read_sdo(['smpl_num_pow2'],[iD])

{696: {'smpl_num_pow2': 5}}

In [ ]:
write_sdo({'smpl_num_pow2': 1},[iD])

In [22]:

write_sdo({'adc_conv_coeff': 1.0},[iD])

{696: {'adc_conv_coeff': 1.0}}

## Load default calib matrix

In [11]:
ctrl_status_cmd_str('LOAD_DFLT_CM', iD)

0x78c1


True

## read 1 channel

In [24]:
read_channel(iD,0)

{'adc_conv': -4.068583, 'adc_raw': 8196}

In [ ]:
read_channel_offset(iD,0,0x7FFF)

In [50]:
read_channel_adc_offset(iD,0,-4)

0x78af


{'adc_conv': -3.696591, 'adc_raw': 8194, 'force_x': 0.301259}

## read all 6 channels once

In [ ]:

adc_conv = list()
adc_raw = list()
for j in range(1):
    for i in range(6):
        yaml_msg = read_channel(iD,i)
        adc_raw.append(yaml_msg['adc_raw'])
        adc_conv.append(yaml_msg['adc_conv'])
        print(yaml_msg)
        time.sleep(0.1)
        
[hex(x) for x in adc_raw]

## reset DAC 

In [12]:
reset_DAC()

{'adc_conv': 0.002745, 'adc_raw': 8209}
{'adc_conv': 0.01586, 'adc_raw': 8295}
{'adc_conv': -0.001067, 'adc_raw': 8184}
{'adc_conv': 0.047885, 'adc_raw': 8505}
{'adc_conv': -0.014945, 'adc_raw': 8093}
{'adc_conv': 0.046665, 'adc_raw': 8497}


## Set all analog channel to Zero using DAC

In [14]:
for ch in range(6) :
    print("--- {0} ---".format(ch))
    for tuned, adc_raw, adc_conv, offset in tune_adc_chan(iD,ch) :
        print("\r%d %f 0x%X 0x%X"%(tuned, adc_conv, adc_raw, offset), end="")
        if tuned :
            write_sdo({'adc_idx': ch,'dac_offset':offset},[iD])
            reply_cmd(ctrl_cmd_set_dac_flash.set_bid(iD))
    print()


--- 0 ---
1 0.000610 0x2003 0x7FDD
--- 1 ---
1 0.000762 0x2004 0x7F3D
--- 2 ---
1 -0.000610 0x1FFB 0x8009
--- 3 ---
1 0.000457 0x2002 0x7D8F
--- 4 ---
1 -0.000762 0x1FFA 0x80B7
--- 5 ---
1 0.000762 0x2004 0x7DA1


In [54]:
for ch in range(6) :
    print("--- {0} ---".format(ch))
    for tuned, adc_raw, adc_conv, offset in tune_adc_offset(iD,ch,0) :
        print("\r%d %f 0x%X %f"%(tuned, adc_conv, adc_raw, offset), end="")
        if tuned :
            pass
            #write_sdo({'adc_idx': ch,'dac_offset':offset},[iD])
            #reply_cmd(ctrl_cmd_set_dac_flash.set_bid(iD))
    print()


--- 0 ---
0x78af
1 -3.689486 0x2003 -3.689486
--- 1 ---
0x78af
1 2.097554 0x2001 2.097554
--- 2 ---
0x78af
1 -2.479642 0x1FF9 -2.479642
--- 3 ---
0x78af
1 0.030064 0x2003 0.030064
--- 4 ---
0x78af
1 0.071006 0x1FF8 0.071006
--- 5 ---
0x78af
1 0.045806 0x2005 0.045806


In [15]:
write_sdo({'dac_tune_bitflag': 0x3F},[iD])
reply_cmd(flash_cmd_save_flash.set_bid(iD))

'Ok !!!'

In [19]:
write_sdo({'smpl_num_pow2': 8},[iD])
reply_cmd(flash_cmd_save_flash.set_bid(iD))

'Ok !!!'

In [ ]:
cnt = 10
dac_offset = 0x7FFF
while (cnt) :
    cnt -= 1
    print(read_channel_offset(iD,0,dac_offset))
    dac_offset += 10


In [28]:
ctrl_status_cmd(0x0012, iD)

0x7812


True

In [29]:
ctrl_status_cmd(0x0021, iD)

0xaa21


False

In [30]:
ctrl_status_cmd(0x00AF, iD)

0x78af


True

In [7]:
reply_cmd(flash_cmd_load_default.set_bid(696))

'Ok !!!'

In [16]:
ctrl_status_cmd_str('LOAD_CM', iD)

0x78c4


True